In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score,confusion_matrix
!pip install easyocr
import matplotlib.pyplot as plt
import cv2
import easyocr

In [ ]:

data_set=pd.read_csv('/content/dataset.csv')
data_set.head()

,Unnamed: 0,languages,language_type,label
0,0,These questions are easy to answer.,english,0
1,1,They are proud of their clever son.,english,0
2,2,They brought trouble on themselves.,english,0
3,3,"This is a good newspaper, isn't it?",english,0
4,4,"To make matters worse, he fell ill.",english,0


In [ ]:
#creating a fuction to clean and stem text 


def text_cleaner(sentence):
  ps = PorterStemmer()
  sentence = re.sub('[^a-zA-Z]', ' ', sentence)
  sentence = sentence.lower()
  sentence = sentence.split()
  sentence = [ps.stem(word) for word in sentence ]
  sentence = ' '.join(sentence)
  return sentence

In [ ]:
#Cleaning the text and stemming the words using nltk library

ps = PorterStemmer()
for i in range(0, len(data_set)):
  data_set['languages'][i]=text_cleaner(data_set['languages'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# shuffling the Data Set rows 

data_shuffled = data_set.sample(frac = 1)

In [ ]:
#creating sample input and sample output for the model

x_sample=data_shuffled['languages']
y_sample=data_shuffled['label ']

In [ ]:
#Creating the sample and test cases

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_sample,y_sample,test_size=1,random_state=0)

In [ ]:
#converting the cases into vectors

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer(5000)
x_trained=cv.fit_transform(x_train).toarray()
x_tested=cv.transform(x_test).toarray()

In [ ]:
#creating the model

from sklearn.naive_bayes import MultinomialNB
language_detector = MultinomialNB().fit(x_trained, y_train)

In [ ]:
#checking the accuracy score of the model

y_predicted=language_detector.predict(x_tested)

print('accuracy of the model is {}'.format(accuracy_score(y_test,y_predicted)))
print('Confusion matrix of the model is {}'.format(confusion_matrix(y_test,y_predicted)))

accuracy of the model is 1.0
Confusion matrix of the model is [[1]]


In [ ]:
#testing the model


text=['शब्दों का ऐसा समूह जिससे पूर्ण भाव व्यक्त हो',
      'Je suis américain.',
      'a pack of cards']

for text_sample in text:
  text_cleaner(text_sample)
  vector=cv.transform([text_sample]).toarray()
  prediction=language_detector.predict(vector)
  

  if prediction[0]==0:
      print('English')
  elif prediction[0]==1:
      print('Hindi')

  else:
      print('french')

Hindi
french
English


In [ ]:
#extracting text from images and detecting its language


def prediction_from_image(input):
    
    reader = easyocr.Reader(['en'])
    
    output = reader.readtext(input,detail=0)

    sentence=' '.join(output)

    text_cleaner(sentence)

    vector=cv.transform([sentence]).toarray()

    prediction=language_detector.predict(vector)

    if prediction[0]==0:
     print('English')
    elif prediction[0]==1:
      print('Hindi')
    else:
      print('French')
    return

In [ ]:

prediction_from_image('/content/English.png')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


English


In [ ]:
prediction_from_image('/content/Hindi.jpg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Hindi


In [ ]:
prediction_from_image('/content/French.jpg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


French
